MIT_AST processing steps, 1 "garden_010120024" folder ata time:

1.define path to folder with .wav files
TODO: 
2.get results from MIT_AST model, results are filename,label,prob_dict
TODO: modify  analyze_files_ast(folder) so the results contain prob_dict
3.speech_detected(), returns true/false, checks if prob_dict contains at least 1 out of 50 speech_related_labels
TODO: 
4.add label, speeech_detected(0/1) columns to metadata file.



In [9]:
import pandas as pd
import os
from tqdm import tqdm
from test_mit_ast import MIT_AST_model
from mit_ast_prob import MIT_AST_model_prob




In [3]:
labels_df = pd.read_excel('MIT_AST_label_map.xlsx')
labels_df.head()

,Unnamed: 0,label,source
0,0,Speech,human
1,1,"Male speech, man speaking",human
2,2,"Female speech, woman speaking",human
3,3,"Child speech, kid speaking",human
4,4,Conversation,human


In [8]:
labels_df['source'].value_counts()

source
music           146
vehicle          51
human            46
animal           36
alarm            31
work             29
material         25
mechanical       22
bird             21
background       21
explosion        19
weather          18
machine          17
foley            16
vocalization     12
door             11
insect            6
Name: count, dtype: int64

In [21]:
# all labels where source human


human = labels_df[labels_df['source']=='human']
print(human)

    Unnamed: 0                                label source
0            0                               Speech  human
1            1            Male speech, man speaking  human
2            2        Female speech, woman speaking  human
3            3           Child speech, kid speaking  human
4            4                         Conversation  human
5            5                 Narration, monologue  human
6            6                             Babbling  human
7            7                   Speech synthesizer  human
8            8                                Shout  human
9            9                               Bellow  human
10          10                                Whoop  human
11          11                                 Yell  human
12          12                           Battle cry  human
13          13                    Children shouting  human
14          14                            Screaming  human
15          15                           Whispering  hum

In [11]:
# extract row with label 'Whistling'

ss = labels_df[labels_df['source']=='bird']
print(ss)

     Unnamed: 0                                    label source
98           98                                     Fowl   bird
99           99                         Chicken, rooster   bird
100         100                                    Cluck   bird
101         101               Crowing, cock-a-doodle-doo   bird
102         102                                   Turkey   bird
103         103                                   Gobble   bird
104         104                                     Duck   bird
105         105                                    Quack   bird
106         106                                    Goose   bird
107         107                                     Honk   bird
111         111                                     Bird   bird
112         112  Bird vocalization, bird call, bird song   bird
113         113                             Chirp, tweet   bird
114         114                                   Squawk   bird
115         115                         

In [4]:



#cfile = 'er_file_2023_09_04_7_13_08.wav'
inhale_file = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_01012024/0/er_file_2024_01_01_11_03_19.wav"
afile ="/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_01012024/16/er_file_2024_01_01_14_18_13.wav"
model_prob = MIT_AST_model_prob()
model = MIT_AST_model()
res = model_prob.classify(inhale_file)
print(res)

Top 5 class/prob: {'Heart murmur': 0.433290958404541, 'Heart sounds, heartbeat': 0.2911057770252228, 'Hum': 0.03401926904916763, 'Throbbing': 0.030839039012789726, 'Stomach rumble': 0.025580795481801033}
{'Animal': 0.14979426562786102, 'Speech': 0.07444065809249878, 'Outside, rural or natural': 0.06758172810077667, 'Clip-clop': 0.06603291630744934, 'Horse': 0.058802176266908646}


In [48]:


def classify_files_ast_prob(filepaths_list):
    """
    Classify a list of audio files using the MIT AST model
    :param filepaths_list: list of file paths
    :return: list of tuples with file path and classification results a list of tuples for each file
            where each tuple contains the file path and a results tuple.
            res tuple contains winning label + dictionary with top 5 labels and their probabilities
            results = [(file_path, (label,{label:prob,...})), ...]
    """
    results = []
    model_prob = MIT_AST_model_prob()
    for file_path in tqdm(filepaths_list, desc="Classifying files"):
        try:
            res = model_prob.classify(file_path)
            if len(res) > 0:
                results.append((file_path, res))
        except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue
    return results
        
      
    

In [15]:
def split_prob_results(prob_results):
    """
    Split the results of the classification into a list of tuples with file path and label and a list of tuples with file path and dictionary
    :param prob_results: list of tuples with file path and classification results dictionary with top labels and their probabilities
                         results = [(file_path, {label: prob, ...}), ...]
    :return: list of tuples with file path and label and a list of tuples with file path and dictionary
    """
    labels = []
    dictionaries = []
    for file_path, label_prob_dict in prob_results:
        # Get the top label
        top_label = max(label_prob_dict, key=label_prob_dict.get)
        labels.append((file_path, top_label))
        dictionaries.append((file_path, label_prob_dict))
    return labels, dictionaries




In [16]:
# human_detected function. finds if human related albels are among the results
def human_detected(prob_results, human_labels):
    """
    Check if any of the labels in the results is a human label and return a list of tuples with file path and a flag
    :param prob_results: list of tuples with file path and classification results dictionary with top 5 labels and their probabilities
                         results = [(file_path, {label: prob, ...}), ...]
    :param human_labels: list of human labels
    :return: list of tuples with file path and 1 if a human label is detected, 0 otherwise
    """
    detected_results = []
    for file_path, label_prob_dict in prob_results:
        # Check if any label is in human_labels
        human_detected = any(label in human_labels for label in label_prob_dict)
        detected_results.append((file_path, 1 if human_detected else 0))
    return detected_results


In [17]:
# add_labels function. Adds a label to the results if it is a human label
def add_labels(metadata_file_path,results,column_name):
    """
    Add MIT_AST labels to the metadata file
    :param metadata_file_path: path to the metadata file
    :param results: list of tuples with file path and top label 
    :param column_name: name of the column to add to the metadata file            
    :return: path to the new metadata file
    """
     # Create a DataFrame from results
    labels_df = pd.DataFrame(results, columns=['filepath', column_name])
    
    # Extract basenames for comparison
    labels_df['filename'] = labels_df['filepath'].apply(os.path.basename)
    
    # Load the existing metadata DataFrame
    df = pd.read_excel(metadata_file_path)
    
    # Ensure filenames are basenames for comparison
    df['filename'] = df['filename'].apply(os.path.basename)
    
    # Check if the 'BirdNET_label' column exists; if not, create it with NaN values
    if column_name not in df.columns:
        df[column_name] = pd.NA
    
    # Merge the existing df with the labels_df
    df = df.merge(labels_df[['filename', column_name]], on='filename', how='left', suffixes=('', '_new'))
    
    # Update the 'MIT_AST_label' column with new values where available
    df[column_name] = df[column_name+'_new'].combine_first(df[column_name])
    
    # Drop the temporary 'BirdNET_label_new' column
    df.drop(column_name+'_new', axis=1, inplace=True)
 
    # Construct the new filename
    original_filename = os.path.basename(metadata_file_path)
    new_filename = original_filename.replace('metadata', column_name + '_metadata')
    new_filepath = os.path.join(os.path.dirname(metadata_file_path), new_filename)
    # Save the updated DataFrame to a new Excel file
    df.to_excel(new_filepath, index=False)
      # Remove the original file
    os.remove(metadata_file_path)
    return new_filepath


In [18]:


def find_files(root_folder):
    """
    Finds the metadata file and audio files in the specified root folder.
    The root folder is assumed to contain subfolders with audio files and metadata file.
    The metadata file is assumed to be named '...metadata.xlsx' and the audio files are assumed to have the extension '.wav'.
    parameters:
    - root_folder: The path to the root folder containing the audio files and metadata file.
    returns:
    - metadata_filepath: The path to the metadata file.
    - audio_files: A list of relative paths to the audio files.

    """
    audio_files = []
    metadata_filepath = None
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('metadata.xlsx'):
                metadata_filepath = os.path.join(dirpath, filename)
                print(f"Found metadata file: {metadata_filepath}")

            elif filename.endswith('.wav'):
                audio_files.append(os.path.join(dirpath, filename))
            

    return metadata_filepath, audio_files

In [19]:


# mit_ast_pipeline
def mit_ast_pipeline(folder_path,human_labels_list):
    """
    Classify all audio files in the folder using the MIT AST model and add the human labels to the metadata file
    :param folder_path: path to the folder with audio files
    :param human_labels_list: list of human labels from MIT_AST_label_map.xlsx
    :return: None
    """
    # Get a list of all audio files in the folder
    metadata_filepath, audio_files = find_files(folder_path)
    
    # Classify the audio files using the MIT AST model
    results = classify_files_ast_prob(audio_files)

    # Split the results into labels and dictionaries
    labels, dictionaries = split_prob_results(results)
    
    # add labels to the metadata file
    new_metadata_filepath = add_labels(metadata_filepath,labels,'MIT_AST')
    print(metadata_filepath)

    # Check if any of the labels in the results is a human label
    human_detected_results = human_detected(dictionaries, human_labels_list)

    # add human detected labels to the metadata file
    add_labels(new_metadata_filepath,human_detected_results,'Human_detected')


   

In [22]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_01012024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)


Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_01012024/garden_01012024_metadata.xlsx


Classifying files: 100%|██████████| 20222/20222 [49:24<00:00,  6.82it/s]   


/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_01012024/garden_01012024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [23]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_02092023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_02092023/garden_02092023_metadata.xlsx


Classifying files: 100%|██████████| 20748/20748 [50:09<00:00,  6.89it/s]  


/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_02092023/garden_02092023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [27]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_02122023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_02122023/garden_02122023_metadata.xlsx


Classifying files:   0%|          | 0/4866 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_02122023/garden_02122023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [28]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_03022024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_03022024/garden_03022024_metadata.xlsx


Classifying files:   0%|          | 0/24276 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_03022024/garden_03022024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [29]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_03032024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_03032024/garden_03032024_metadata.xlsx


Classifying files:   0%|          | 0/8567 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_03032024/garden_03032024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [ ]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_06012023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

In [32]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_07122023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_07122023/garden_07122023_metadata.xlsx


Classifying files:   0%|          | 0/25617 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_07122023/garden_07122023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [34]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_08032024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_08032024/garden_08032024_metadata.xlsx


Classifying files:   0%|          | 0/25809 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_08032024/garden_08032024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [35]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_10092023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_10092023/garden_10092023_metadata.xlsx


Classifying files:   0%|          | 0/12696 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_10092023/garden_10092023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [36]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_11082023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_11082023/garden_11082023_metadata.xlsx


Classifying files:   0%|          | 0/4823 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_11082023/garden_11082023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [37]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_12112023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_12112023/garden_12112023_metadata.xlsx


Classifying files:   0%|          | 0/32772 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_12112023/garden_12112023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [39]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_16032024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_16032024/garden_16032024_metadata.xlsx


Classifying files:   0%|          | 0/24671 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_16032024/garden_16032024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [50]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_16122023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_16122023/garden_16122023_metadata.xlsx


Classifying files:   0%|          | 0/31574 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_16122023/garden_16122023_metadata.xlsx


In [42]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_19102023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_19102023/garden_19102023_birdnet_metadata.xlsx


Classifying files:   0%|          | 0/1619 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_19102023/garden_19102023_birdnet_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [51]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_20012024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_20012024/garden_20012024_metadata.xlsx


Classifying files:   0%|          | 0/45069 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_20012024/garden_20012024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [44]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_21092023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_21092023/garden_21092023_metadata.xlsx


Classifying files:   0%|          | 0/20886 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_21092023/garden_21092023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [45]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_21102023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_21102023/garden_21102023_metadata.xlsx


Classifying files:   0%|          | 0/17620 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_21102023/garden_21102023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [49]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/garden_23122023_metadata.xlsx


Classifying files:   0%|          | 0/15162 [00:00<?, ?it/s]

Error processing file /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/146/er_file_2023_12_24_20_28_23.wav: Failed to open the input "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/146/er_file_2023_12_24_20_28_23.wav" (Invalid data found when processing input).
Error processing file /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/146/er_file_2023_12_24_20_55_23.wav: Failed to open the input "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/146/er_file_2023_12_24_20_55_23.wav" (Invalid data found when processing input).
Error processing file /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/146/er_file_2023_12_24_21_47_54.wav: Failed to open the input "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_23122023/146/er_file_2023_12_24_21_47_54.wav" (Invalid data found when processing input).
Error processing file /Users/evgenynazarenko/DACS_3_year/Thesis/Garden

/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [38]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_25112023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_25112023/garden_25112023_metadata.xlsx


Classifying files:   0%|          | 0/5257 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_25112023/garden_25112023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [52]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_26012024"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_26012024/garden_26012024_metadata.xlsx


Classifying files:   0%|          | 0/29225 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_26012024/garden_26012024_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


In [54]:
# test pipeline
garden_folder = "/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_30102023"
human_labels = list(human['label'])
mit_ast_pipeline(garden_folder,human_labels)

Found metadata file: /Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_30102023/garden_30102023_metadata.xlsx


Classifying files:   0%|          | 0/3361 [00:00<?, ?it/s]

/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23/garden_30102023/garden_30102023_metadata.xlsx


/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_81228/1414391559.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[column_name] = df[column_name+'_new'].combine_first(df[column_name])


create dull meatdata file 

In [70]:
# read the metadata file
df_meta_2504 = pd.read_excel('metadata/garden_concat_metadata_birdnet_labeled_mit_ast_2404.xlsx')
df_meta_2504.shape

(406689, 27)

In [71]:
df_meta_2504.head()

,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,0.255452,0.50,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,NaN,Bird,bird
1,1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,0.271969,0.50,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
2,2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,0.023406,0.24,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
3,3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,0.172581,0.24,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird
4,4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,0.044865,0.24,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,NaN,Bird,bird


In [65]:
# get metadata df from files that ends with 'metadata.xlsx'  from all subfolders of the GardenFiles23 folder
def get_metadata_files(folder_path):
    """
    Get metadata  from all subfolders of the specified folder
    :param folder_path: path to the folder
    :return: metadata_df: DataFrame with metadata

    """
    metadata_df = pd.DataFrame()
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('metadata.xlsx'):
                # read the metadata file
                meta_df = pd.read_excel(os.path.join(dirpath, filename))
                metadata_df = pd.concat([metadata_df, meta_df], ignore_index=True)
    return metadata_df






In [66]:
# get metadata from all subfolders of the GardenFiles23 folder
metadata_human_det_df = get_metadata_files('/Users/evgenynazarenko/DACS_3_year/Thesis/GardenFiles23')
metadata_human_det_df.shape

(406869, 27)

In [67]:
metadata_human_det_df.head()

,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,MIT_AST,Human_detected,BirdNET_label
0,0.0,test,_2024_01_06_11_20_56,er_file_2024_01_06_11_20_56.wav,0.164558,0.32,0.673863,0.5,0.215897,0.50,...,3.9,94.0,170.0,0.0,35.1,5.462251,51.417782,Caw,1.0,NaN
1,1.0,test,_2024_01_06_11_21_01,er_file_2024_01_06_11_21_01.wav,0.225941,0.20,0.496771,0.5,0.023409,0.21,...,3.9,94.0,170.0,0.0,35.1,5.462251,51.417782,Speech,1.0,NaN
2,2.0,test,_2024_01_06_11_21_04,er_file_2024_01_06_11_21_04.wav,0.145084,0.20,0.672597,0.5,0.039366,0.21,...,3.9,94.0,170.0,0.0,35.1,5.462251,51.417782,Caw,1.0,NaN
3,3.0,test,_2024_01_06_11_21_07,er_file_2024_01_06_11_21_07.wav,0.150078,0.20,0.778181,0.5,0.079053,0.21,...,3.9,94.0,170.0,0.0,35.1,5.462251,51.417782,Mouse,1.0,NaN
4,4.0,test,_2024_01_06_11_21_10,er_file_2024_01_06_11_21_10.wav,0.142194,0.20,0.792797,0.5,0.024214,0.21,...,3.9,94.0,170.0,0.0,35.1,5.462251,51.417782,Bird,1.0,NaN


In [72]:
# add Human_detected from metadata_human_det_df to  df_meta_2504 on the 'filename' column
df_meta_2504['Human_detected'] = metadata_human_det_df['Human_detected']
df_meta_2504.head()

,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,...,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label,Human_detected
0,0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,0.255452,0.50,...,88.0,286.0,0.0,36.0,5.462251,51.417782,NaN,Bird,bird,1.0
1,1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,0.271969,0.50,...,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird,1.0
2,2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,0.023406,0.24,...,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird,1.0
3,3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,0.172581,0.24,...,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird,1.0
4,4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,0.044865,0.24,...,88.0,286.0,0.0,36.0,5.462251,51.417782,NaN,Bird,bird,1.0


In [74]:
#drop Unnamed: 0 column
df_meta_2504.drop('Unnamed: 0', axis=1, inplace=True)

In [73]:
df_meta_2504['Human_detected'].value_counts()

Human_detected
0.0    254568
1.0    135549
Name: count, dtype: int64

In [75]:
df_meta_2504.to_excel('metadata_birdnet_mit_ast_human_detected_columns_18062024.xlsx', index=False)